In [1]:
import os
import re
import fitz
import glob
import nltk
import cv2
import spacy
import spacy.cli
from spacy.language import Language
from spacy_langdetect import LanguageDetector
import easyocr
import pytesseract
import numpy as np
import textseg as ts
from spacy import displacy
from PyPDF2 import PdfFileReader
from pdf2image import convert_from_path

# nltk.download('punkt')
# nltk.download('stopwords')
# spacy.cli.download("en_core_web_lg")

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe' #tesseract.exe location in your computer

In [9]:
# SKILLS EXTRACTION
# Add skills database from a file
def add_skills_data(filePath):
    skills = []

    for data in open(filePath, 'r', encoding='UTF-8'):
        skills.append(data.strip())

    return skills

# Get the text from a file
def extract_text(filePath, remove_line=False):
    with fitz.open(filePath) as doc:
        text = ""
        for page in doc:
            text += page.get_text()

        if remove_line:
            text = text = re.sub('\s', " ", text)

    return text

# Extract the skills based on the skill database
def extract_skills(input_text, skills_data):
    stop_word = set(nltk.corpus.stopwords.words('english'))
    word_tokens = nltk.tokenize.word_tokenize(input_text)

    filtered_tokens = [w for w in word_tokens if w not in stop_word]
    filtered_tokens = [w for w in word_tokens if w.isalpha()]
    bigrams_trigrams = list(map(' '.join, nltk.everygrams(filtered_tokens, 2, 3)))

    skills = set() #penampung skill

    for token in filtered_tokens:
        if token in skills_data:
            skills.add(token)

    for ngram in bigrams_trigrams:
        if ngram in skills_data:
            skills.add(ngram)

    return skills

# Extract skills from a single file
def extract_single_skills(filePath, skills):
    text = extract_text(filePath)

    return  extract_skills(text, skills)

# Extract skills from a folder full of pdf
def extract_batch_skill(filePath, skills):
    data = {"File": [], 'Skill': []}

    for file in glob.glob('{}*.pdf'.format(filePath)):
        text = extract_text(file, True)
        data['File'].append(file)
        data['Skill'].append(extract_skills(text, skills))

    return data

# DOCUMENT SEGMENTATION
# Converting from pdf to image for segmentation
def convert_pdf_to_image(filepath,img_path_to_save):
    try:
        fileName = filepath.split("/")[-1].replace(".pdf","")
        pages = convert_from_path(filepath, 350)
        i = 1
        for page in pages:
            image_name = img_path_to_save+fileName+"Page_" + str(i) + ".png"
            page.save(image_name, "JPEG")
            i = i+1
        return {"status":200,"response":"PDF Converted to image sucessfully","fileName":fileName}
    except Exception as e:
        return {"status":400,"response":str(e)}

# Extract text from a png
def text_from_tesseract(output_img):
    text = str(((pytesseract.image_to_string(output_img))))
    return text

def text_from_easyocr(img, reader):
    all_text = ""
    result = reader.readtext(img)

    for (bbox, text, prob) in result:
        all_text += text + " "

    return all_text


# Segment and then extract the data from a resume
def segment_extract_data(data,  path_to_write, reader, singleFile=True):
    documents = [] # file path nya untuk pdf

    if singleFile:
        documents.append(data)
    else:
        documents = data

    final_name_list=[] # nama file
    final_text_opencv=[] # text dengan segmen
    # final_text_tessaract=[]
    final_text_easyocr=[] # semua text tanpa segmen
    for i in documents:
        pdf = PdfFileReader(open(i,'rb'))
        fname = i.split('/')[-1]

        # if pdf.getNumPages() > 3:
        #     print('Pages to many : {}!, Skipping...'.format(pdf.getNumPages()))
        #     continue

        images = convert_from_path(i)
        resumes_img=[]
        for j in range(len(images)):
            # Save pages as images in the pdf
            images[j].save(path_to_write+fname.split('.')[0]+'_'+ str(j) +'.jpg', 'JPEG')
            resumes_img.append(path_to_write+fname.split('.')[0]+'_'+ str(j) +'.jpg')
        name_list = fname.split('.')[0]+'_' +'.jpg'
        text_opencv=[]
        # text_tessaract=[]
        text_easyocr=[]
        for i in resumes_img:
            frame=cv2.imread(i)
            os.remove(i)
            img = i.split("/")[2]

            output_img,label,dilate, c_dict,df1, split_img=ts.get_text_seg(frame, img)
            cv2.imwrite(path_to_write+img.split('.')[0]+".png",output_img)
            for i in range(len(split_img)):
                cv2.imwrite(path_to_write+img.split('.')[0]+str(i)+".png", split_img[i])

            text_opencv.append(c_dict)
            # text_tessaract+=text_from_tesseract(output_img)
            # tesseract_str = ''.join(text_tessaract)
            text_easyocr+=text_from_easyocr(output_img, reader)
            easyocr_str = ''.join(text_easyocr)

        final_name_list.append(name_list)
        final_text_opencv.append(text_opencv)
        # final_text_tessaract.append(tesseract_str)
        final_text_easyocr.append(easyocr_str)

    return final_text_opencv, final_name_list, final_text_easyocr

# EXPERIENCE EXTRACTION
# Extract exp from a text
def extract_exp(textList, nlp):
    exp = []

    for i in range(len(textList)):
        for j in range(len(textList[i])):
            for _, text in textList[i][j].items():
                text = re.sub(r'[^\w\s]+', "", text)
                text = re.sub(r'[\s]{2,}', " ", text)
                text = re.sub(r'https\w+', "", text)
                doc = nlp(text)
                if doc.cats['experience'] > 0.70:
                    exp.append(text)

    return exp

# Do all the above with just 1 function
def extract_data(filePath, skills, nlp, temp_path, reader):
    file_data = {'File': "", 'Skills':"", "Exp":""}

    textList, fileName, fullText = segment_extract_data(filePath, temp_path, reader)
    file_data['File'] = fileName[0]
    file_data['Skills'] = extract_skills((fullText[0]), skills_data=skills)
    file_data['Exp'] = extract_exp(textList, nlp)

    return file_data

def batch_extract_data(filePath, skills, nlp, temp_path):
    file_data = {'File': [], 'Skills': [], "Exp": []}

    for file in os.listdir(filePath):
        data = extract_data('{}/{}'.format(filePath, file), skills, nlp, temp_path)
        file_data['File'].append(data['File'])
        file_data['Skills'].append(data['Skills'])
        file_data['Exp'].append(data['Exp'])

    return file_data

In [10]:
# Adding skills database
skills = add_skills_data('list_of_skills.txt')
skills[0] = '.NET' # First skills is not UTF-8 so we need to replace it

# Segmentation need a temp folder for storing image that will be scanned for extraction the text
temp_path = ('./segmentation/')

# Load the machine learning model for exp classification
nlp = spacy.load('model/model_exp')

reader = easyocr.Reader(['en'])

# Get the filename, skills, exp
data = extract_data('./sample/cv/Alya Ataya_CV - Alya Ataya.pdf', skills, nlp, temp_path, reader)

data

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [4]:
import pandas as pd

df = pd.read_csv('sample/data.csv')

for i in range(1,11):
    try:
        data = extract_data("sample/analysis/{}.pdf".format(i), skills, nlp, temp_path, reader)
        df.at[i, 'key'] = data['Exp']
    except:
        print("IDK can't extract")

Using CPU device


Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.


Using CPU device


Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.


Using CPU device


...

Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.


Using CPU device


Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.


Using CPU device


Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with a GPU.
Using CPU. Note: This module is much faster with

In [18]:
df = df.loc[0:11, :]
df

,Email,key,Nama Peserta,Project,MD,MI,MS,MC,LD,LI,...,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
1,atkamliaa@gmail.com,[SUMMARY When was in college I was experienced...,Atikah Amalia,Hiring Batch 1 2,7,3,2,6,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,deri45anggraini@gmail.com,[PT_ Bank Negara Indonesia 20202021 Job descri...,Deri Firma Anggraini,Hiring Batch 1 2,8,2,3,7,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,elisabethda1994@gmail.com,[COMMITTEES Coordinator of the public relation...,Elisabeth Dwi Anggraeni,Hiring Batch 1 2,8,3,2,5,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jordyrevindra@gmail.com,[PT Pertamina RU IV Persero Cilacap Jawa Tenga...,Jordy Revindra Priana,Hiring Batch 1 2,5,4,3,7,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,nisakusuma20@gmail.com,[EXPERIENCE PT Metranet PaDi UMKM Telkom Group...,Nisa Kusuma Febri,Hiring Batch 1 2,4,3,5,7,6,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,nugroholulut@gmail.com,[],Lulut Nugroho,Hiring Batch 1 2,3,4,2,7,5,8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,adhibhawika@gmail.com,[SENAT MAHASISWA UNIVERSITAS DIPONEGORO JAN 20...,Adhi Bhawika Prabhaswara,Hiring Batch 1 3,7,3,0,6,4,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,c.chusnulkotimah@gmail.com,[Experienced and energetic Marketing Manager w...,Chusnul Kotimah,Hiring Batch 1 3,5,0,4,4,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,devisarasaty@gmail.com,[KERJAINPLIS Freelance Writer April 2021 Prese...,Devi Sarasaty,Hiring Batch 1 3,9,2,0,6,3,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,fkevinirwanto@gmail.com,[October 2019 PT Angkasa Pura 1 Persero Human ...,Kevin Irwanto,Hiring Batch 1 3,8,1,3,5,1,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
dfa = df.drop([6, 11])

In [27]:
dfa

,Email,key,Nama Peserta,Project,MD,MI,MS,MC,LD,LI,...,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71,Unnamed: 72,Unnamed: 73,Unnamed: 74,Unnamed: 75,Unnamed: 76,Unnamed: 77
1,atkamliaa@gmail.com,[SUMMARY When was in college I was experienced...,Atikah Amalia,Hiring Batch 1 2,7,3,2,6,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,deri45anggraini@gmail.com,[PT_ Bank Negara Indonesia 20202021 Job descri...,Deri Firma Anggraini,Hiring Batch 1 2,8,2,3,7,4,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,elisabethda1994@gmail.com,[COMMITTEES Coordinator of the public relation...,Elisabeth Dwi Anggraeni,Hiring Batch 1 2,8,3,2,5,4,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,jordyrevindra@gmail.com,[PT Pertamina RU IV Persero Cilacap Jawa Tenga...,Jordy Revindra Priana,Hiring Batch 1 2,5,4,3,7,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,nisakusuma20@gmail.com,[EXPERIENCE PT Metranet PaDi UMKM Telkom Group...,Nisa Kusuma Febri,Hiring Batch 1 2,4,3,5,7,6,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,adhibhawika@gmail.com,[SENAT MAHASISWA UNIVERSITAS DIPONEGORO JAN 20...,Adhi Bhawika Prabhaswara,Hiring Batch 1 3,7,3,0,6,4,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,c.chusnulkotimah@gmail.com,[Experienced and energetic Marketing Manager w...,Chusnul Kotimah,Hiring Batch 1 3,5,0,4,4,6,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,devisarasaty@gmail.com,[KERJAINPLIS Freelance Writer April 2021 Prese...,Devi Sarasaty,Hiring Batch 1 3,9,2,0,6,3,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,fkevinirwanto@gmail.com,[October 2019 PT Angkasa Pura 1 Persero Human ...,Kevin Irwanto,Hiring Batch 1 3,8,1,3,5,1,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
def a(t):
    text = ""
    for i in t:
        text += i + " "

    return text

dfa['key'] = dfa.key.apply(lambda x: a(x))

In [32]:
dfa.to_csv('exp_train.csv')

In [2]:
import fitz
import spacy
import spacy.displacy

nlp = spacy.load('model/model_job')

def extract_text(filePath, remove_line=False):
    with fitz.open(filePath) as doc:
        finaltext = ""
        for page in doc:
            text = page.get_text()
            text = text.replace("\n"," ")
            text = text.replace("[^a-zA-Z0-9]", " ");
            text = re.sub('\W+',' ', text)
            text = re.sub('[^A-Za-z0-9]',' ', text)
            finaltext += text

        if remove_line:
            finaltext = text = re.sub('\s', " ", text)

    return finaltext

text = extract_text('./sample/cv/Curriculum Vitae Zaky (ver2 Oct 2021) - Ahmad Zaky Nabhan.pdf')

doc = nlp(text)

displacy.render(doc, style='ent')

C:\Users\sakur\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')



In [20]:
import re

job = set()

for a in doc.ents:
    job.add(str(a).lower())

job

{'database administration',
 'hc specialist',
 'human capital information system',
 'human capital staff',
 'people development',
 'recruitment assessment staff',
 'senior hc staff',
 'talent monitoring'}

In [18]:
import spacy

nlp = spacy.load('en_core_web_sm')

doc = nlp("Steven like to eat in Google")

for token in doc.ents:
    print(token.text, token.lemma_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

AttributeError: 'spacy.tokens.span.Span' object has no attribute 'dep_'